In [1]:
import os, sys
sys.path.append(os.getcwd()+"/..")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colorbar
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline
os.getcwd()

'C:\\Users\\benja\\OneDrive - ETHZ\\PhD\\Code\\ensembler_public\\examples'

In [2]:
import ensembler.potentials.OneD as pot
import ensembler.system.perturbed_system as system
import ensembler.ensemble.replicas_dynamic_parameters as cvb
from ensembler.integrator import stochastic


SYSTEM ASSIGNS random POSITION FOR: states: 1	nDim: 1


# Interactive Example 

In [3]:
from ensembler.visualisation.interactive_plots import interactive_conveyor_belt

iwidget = interactive_conveyor_belt(nbins=10, numsys=3, steps=10)

ModuleNotFoundError: No module named 'ensembler.visualisation.plotConveyorBelt.plotEnsembler'; 'ensembler.visualisation.plotConveyorBelt' is not a package

# BUILD Conveyor Belt

In [ ]:
numsys = 8
steps = 10

integrat = stochastic.metropolisMonteCarloIntegrator()
potial = pot.linearCoupledPotentials()
syst = system.perturbedSystem(potential=potial , integrator=integrat)

In [ ]:
ens=cvb.ConveyorBelt(0.0, numsys, system=syst, build=False)
ens


In [ ]:
#ens.traj_clear()
ens.simulate(steps)

print(ens.reject)


In [ ]:
(cvb_traj, systrajs) = ens.get_trajs()

In [ ]:
systrajs[0].head()

In [ ]:
# old: traj=pd.DataFrame(traj, columns=['Step', 'CapLam', 'Energy', 'Bias'])
cvb_traj.head()

# Plot Simulation

In [ ]:
nbins=100
bins=np.zeros(nbins)
dhdlbins=np.zeros(nbins)
for i in systrajs:
    for j in range(systrajs[i].shape[0]):
        index=int(np.floor(systrajs[i].lam[j]*nbins))
        if index == nbins:
            index=nbins-1
        bins[index]+=1
        dhdlbins[index]+=systrajs[i].dhdlam[j]
dhdlbins/=bins
#dhdlbins

In [ ]:
plt.plot(np.linspace(0,1,nbins), dhdlbins)
integral=np.sum(dhdlbins)*1.0/nbins
print('Integral = ', integral)

In [ ]:
h1=np.histogram(cvb_traj.capital_lambda, bins=50, density=1)
plt.plot(h1[1][:-1], h1[0])
#plt.ylim(0.14, 0.185)

In [ ]:
caplam = np.linspace(0, 2*np.pi, 100)
x=[]
for i in caplam:
    x.append(ens.calc_lam(i, 0))
plt.plot(caplam, x)
plt.xlabel('$\Lambda / \pi$')
plt.ylabel('$\lambda$')

In [ ]:
#analytical
u=1.66053886e-27
NA=6.0221415e23
hbar=1.054571800e-34*1e12*1e-3*NA  #kJ/mol*ps
R=0.00831446 #kJ/mol/K
mu=0.5  #u
T=300.0  #K
fc1=1  #kJ/nm^2/mol
fc2=11.0 #kJ/nm^2/mol
omega1=np.sqrt(fc1/mu)
omega2=np.sqrt(fc2/mu)
print('Omega', omega1, omega2)
alpha1=hbar*np.sqrt(fc1/mu)/(R*T)
alpha2=hbar*np.sqrt(fc2/mu)/(R*T)
print(alpha1, alpha2)
Z1=np.exp(-alpha1/2.0)/(1-np.exp(-alpha1))
Z2=np.exp(-alpha2/2.0)/(1-np.exp(-alpha2))
F1=-R*T*np.log(Z1)
F2=-R*T*np.log(Z2)
DF=F2-F1
DF2=-R*T*np.log(Z2/Z1)
print(DF, DF2, F1, F2)

# Conveyor Belt Animation

In [ ]:
import ensembler.visualisation.plotConveyorBelt as plotEnsembler
nbins=2
dhdlbins=2
lam = np.linspace(0, 1, nbins)
ene = np.cumsum(dhdlbins)/nbins

fig = plotEnsembler.drawFunicular(lam, ene, CapLam=0.2, M=8)